In [3]:
import tensorflow as tf
import tensorflow.contrib.layers as layers

import functools
import os
import pickle

import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [4]:
dir_path = os.getcwd()

In [5]:
#load embendings

embeddings = np.loadtxt(os.path.join(dir_path,'cache','embeddings.vec'))

pickle_in = open('cache/database.dict', 'rb')
db = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('cache/traindata.list', 'rb')
traindata = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('cache/data2onehot.dict', 'rb')
dictionary = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
from JavaClassParser import ByteCode
reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))
X_train=[]
Y_train=[]
X_train_long=[]
Y_train_long=[]
X_labels = []
Y_labels = []

for dclass in db.values():
    for method in dclass.values():
        instructions = method['x']
        labels = method['y']
        byteIndex = method['index']
        
        
        #
        #seperating by labels

        cur_section = []
        cur_label = labels[0]
        if len(instructions)<100:
            X_train.append(instructions)
            Y_train.append(labels)
        else:
            X_train_long.append(instructions)
            Y_train_long.append(labels)
        for instruction, label in zip(instructions,labels):
            if label != cur_label:
                X_labels.append(cur_section)
                Y_labels.append(cur_label)
                cur_section = []
                cur_label = label
            cur_section.append(dictionary.get(instruction,0))

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_train_long = np.array(X_train_long)
Y_train_long = np.array(Y_train_long)
X_labels = np.array(X_labels)
Y_labels = np.array(Y_labels)

In [ ]:
class Conv:
    def __init__(self, 
                 learning_rate=1e-1, 
                 num_epochs=10,
                 weight_decay = 1e-2,
                 conv1sz = 16,
                 conv2sz = 32,
                 fc3sz = 512,
                 num_classes = 2):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weight_decay = 1e-2
        self.conv1sz = conv1sz = 16
        self.conv2sz = conv2sz = 32
        self.fc3sz = fc3sz = 
        self.num_classes = num_classes 
        

def cnn_model(features, labels, mode):
      weight_decay = self.weight_decay
      conv1sz = self.conv1sz
      conv2sz = self.conv2sz
      fc3sz = self.fc3sz
      num_classes = self.num_classes
      inputs = tf.reshape(features["x"], [-1, 28, 28, 1])
    with tf.contrib.framework.arg_scope([layers.convolution2d],
          kernel_size=5, stride=1, padding='SAME', activation_fn=tf.nn.relu,
          weights_initializer=layers.variance_scaling_initializer(),
          weights_regularizer=layers.l2_regularizer(weight_decay)):

        net = layers.convolution2d(inputs, conv1sz, scope='conv1')
        net = layers.max_pool2d(net, 2, 2, scope='pool1')
        net = layers.convolution2d(net, conv2sz, scope='conv2')
        net = layers.max_pool2d(net, 2, 2, scope='pool2')
        inputs  = net

      with tf.contrib.framework.arg_scope([layers.fully_connected],
          activation_fn=tf.nn.relu,
          weights_initializer=layers.variance_scaling_initializer(),
          weights_regularizer=layers.l2_regularizer(weight_decay)):

        net = layers.flatten(inputs)
        net = layers.fully_connected(net, fc3sz, scope='fc3')

      logits = layers.fully_connected(net, num_classes, activation_fn=None, scope='logits')
      #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(lables=labels, logits=logits))

      predictions = {
          # Generate predictions (for PREDICT and EVAL mode)
          "classes": tf.argmax(input=logits, axis=1),
          # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
          # `logging_hook`.
          "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
      }

      if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

      # Calculate Loss (for both TRAIN and EVAL modes)
      onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
      loss = tf.losses.softmax_cross_entropy(
          onehot_labels=onehot_labels, logits=logits)

      # Configure the Training Op (for TRAIN mode)
      if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

      # Add evaluation metrics (for EVAL mode)
      eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"])}
      return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [ ]:

def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if not arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator


@doublewrap
def define_scope(function, scope=None, *args, **kwargs):
    """
    A decorator for functions that define TensorFlow operations. The wrapped
    function will only be executed once. Subsequent calls to it will directly
    return the result so that operations are added to the graph only once.
    The operations added by the function live within a tf.variable_scope(). If
    this decorator is used with arguments, they will be forwarded to the
    variable scope. The scope name defaults to the name of the wrapped
    function.
    """
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return decorator


class Model:

    
    def __init__(self, image, label,
                 learning_rate=1e-1, 
                 num_epochs=10,
                 weight_decay = 1e-2,
                 conv1sz = 16,
                 conv2sz = 32,
                 fc3sz = 512):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weight_decay
        self.conv1sz = conv1sz
        self.conv2sz = conv2sz
        self.fc3sz = fc3sz
        
        self.image = image
        self.label = label
        
        self.num_instructions = tf.shape(label)[-1]
        
        self.prediction
        self.optimize
        self.error

    @define_scope
    def prediction(self):
        inputs = self.image
        with tf.contrib.framework.arg_scope([layers.convolution2d],
          kernel_size=5, stride=1, padding='SAME', activation_fn=tf.nn.relu,
          weights_initializer=layers.variance_scaling_initializer(),
          weights_regularizer=layers.l2_regularizer(weight_decay)):

            net = layers.convolution2d(inputs, conv1sz, scope='conv1')
            net = layers.max_pool2d(net, 2, 2, scope='pool1')
            net = layers.convolution2d(net, conv2sz, scope='conv2')
            net = layers.max_pool2d(net, 2, 2, scope='pool2')
            inputs  = net
            
        with tf.contrib.framework.arg_scope([layers.fully_connected],
          activation_fn=tf.nn.relu,
          weights_initializer=layers.variance_scaling_initializer(),
          weights_regularizer=layers.l2_regularizer(weight_decay)):

            net = layers.flatten(inputs)
            net = layers.fully_connected(net, fc3sz, scope='fc3')
        
        instr_predict = layers.fully_connected(net, self.num_instructions, activation_fn=None, scope='instructions')
        predict = tf.nn.sigmoid(instr_predict, name='predict')
        return predict

    @define_scope
    def optimize(self):
        logprob = tf.log(self.prediction + 1e-12)
        cross_entropy = -tf.reduce_sum(self.label * logprob)
        optimizer = tf.train.RMSPropOptimizer(0.03)
        return optimizer.minimize(cross_entropy)

    @define_scope
    def error(self):
        mistakes = tf.not_equal(
            tf.argmax(self.label, 1), tf.argmax(self.prediction, 1))
        return tf.reduce_mean(tf.cast(mistakes, tf.float32))


def main():
    mnist = input_data.read_data_sets('./mnist/', one_hot=True)
    image = tf.placeholder(tf.float32, [None, 784])
    label = tf.placeholder(tf.float32, [None, 10])
    model = Model(image, label)
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())

    for _ in range(10):
      images, labels = mnist.test.images, mnist.test.labels
      error = sess.run(model.error, {image: images, label: labels})
      print('Test error {:6.2f}%'.format(100 * error))
      for _ in range(60):
        images, labels = mnist.train.next_batch(100)
sess.run(model.optimize, {image: images, label: labels})